# Main

Objetivo: Utilizar os algoritmos de otimização implementados para encontrar o(s) ponto(s) de mínimo da função objetivo $$f(x) = \sqrt{\ln((\prod_{i=1}^{5}x_i(1-x_i))+1)},$$ onde x = (x_1, x_2, x_3, x_4, x_5).

### Observações

Devido a problemas de precisão numérica otimizando a função objetivo f(x) (underflow), foi decidido otimizar uma função alternativa $$g(x) = (\prod_{i=1}^{5}x_i(1-x_i))+1,$$ que é equivalente a minimizar a função f(x) pois as funções raíz quadrada e logaritmo são funções não-decrescentes.

Além disso, como a função objetivo só está definida quando $$(\prod_{i=1}^{5}x_i(1-x_i))+1 \geq 1 \\ \Leftrightarrow \prod_{i=1}^{5}x_i(1-x_i) \geq 0 \\ \Leftrightarrow -\prod_{i=1}^{5}x_i(1-x_i) \leq 0,$$ pois a função raíz quadrada requer que o argumento seja maior ou igual a zero, e durante o algoritmo pode ser que o argumento deixe de satisfazer a esta inequação, foi decidido utilizar uma função de penalização $$p(x) = \rho \max(0, -\prod_{i=1}^{5}x_i(1-x_i))^2,$$ com parâmetro rho = 0.001.

## Bibliotecas

In [202]:
import random
import numpy as np
import pandas as pd

from algorithms import Gradiente, Newton_1D, Newton_ND, BFGS

from obj_test_util3_pn import f, gradient_f, hessian_f
from objective_function_util import f as obj_fn

## Aplicando algoritmos

### Gerando pontos iniciais

In [203]:
def prod(sample):
    out = 1
    for i in range(0, 5):
        out = out * sample[i] * (1 - sample[i])
    return out

def gen_sample():
    sample = []
    for i in range(0, 5):
        sample.append(random.uniform(0, 1))
    while prod(sample) < 0:
        sample = []
        for i in range(0, 5):
            sample.append(random.uniform(0, 1))
    return sample

In [204]:
initial_points = []

for i in range(0, 5):
    initial_points.append(gen_sample())

for initial_point in initial_points:
    print(initial_point)

[0.5218226579138382, 0.1766351505210949, 0.5230897881307102, 0.006077009838843517, 0.24398081516009718]
[0.5156780040614635, 0.956428224641158, 0.8693971398531677, 0.615504143527244, 0.23086015479926747]
[0.2946226807749026, 0.3898831682375272, 0.2645512612918487, 0.7115753853206658, 0.7818204400208536]
[0.34438551450511534, 0.7289656385061903, 0.35330868392429093, 0.5075578743337829, 0.6734455167549697]
[0.13211349436008835, 0.9989410860061092, 0.8374395095548139, 0.7823695821889023, 0.8345965420536839]


### Método do Gradiente

Foi utilizado como critério de parada $$\Vert \nabla f(x_k) \Vert \leq \epsilon$$ ou que k iterações do algoritmo tenham se passado. Onde x_k é o ponto x na k-ésima iteração, o valor de epsilon é fixado como 10^-6 e k é um parâmetro do algoritmo, que nos experimentos foi utilizado o valor k = 100.

In [205]:
k = 100
data = []

for i in range(0, 5):
    x0 = initial_points[i]
    x, iterations, armijo_calls, error = Gradiente(f, gradient_f, x0, k)
    data.append([x0, iterations, armijo_calls, x, obj_fn(x), error])

columns = ["Ponto inicial", "# de iterações", "# de chamadas de Armijo", "Ponto ótimo", \
"Valor ótimo", "Erro de aproximação"]

df_gradiente = pd.DataFrame(data, columns=columns)

/var/home/pc-146/Documents/UFRJ/8° Período/Otimização/Trabalhos/Trabalho Prático/cos360_optimization_implementation_assignment/objective_function_util.py:7: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.log(prod + 1))


### Método de Newton

Foi utilizado como critério de parada $$\frac{\Vert \nabla f(x_{k-1}+td) - \nabla f(x_{k-1}) \Vert}{\Vert td \Vert} \leq \epsilon$$ ou que k iterações do algoritmo tenham se passado. Onde x_{k-1} é o ponto x obtido na (k-1)-ésima iteração, t é o tamanho do passo, d é a direção do passo, o valor de epsilon é fixado como 10^-6 e k é um parâmetro do algoritmo, que nos experimentos foi utilizado o valor k = 100.

In [206]:
k = 100
data = []

for i in range(0, 5):
    x0 = initial_points[i]
    x, iterations, error = Newton_ND(f, gradient_f, hessian_f, x0, k)
    data.append([x0, iterations, x, obj_fn(x), error])

columns = ["Ponto inicial", "# de iterações", "Ponto ótimo", \
"Valor ótimo", "Erro de aproximação"]

df_newton = pd.DataFrame(data, columns=columns)

### Método Quase-Newton (BFGS)

Foi utilizado como critério de parada $$\frac{\Vert \nabla f(x_{k-1}+td) - \nabla f(x_{k-1}) \Vert}{\Vert td \Vert} \leq \epsilon$$ ou que k iterações do algoritmo tenham se passado. Onde x_{k-1} é o ponto x obtido na (k-1)-ésima iteração, t é o tamanho do passo, d é a direção do passo, o valor de epsilon é fixado como 10^-6 e k é um parâmetro do algoritmo, que nos experimentos foi utilizado o valor k = 100.

In [207]:
k = 100
dimensions = 5
data = []

for i in range(0, 5):
    x0 = initial_points[i]
    x, iterations, error = BFGS(f, gradient_f, x0, dimensions, k)
    data.append([x0, iterations, x, obj_fn(x), error])

columns = ["Ponto inicial", "# de iterações", "Ponto ótimo", \
"Valor ótimo", "Erro de aproximação"]

df_bfgs = pd.DataFrame(data, columns=columns)

/var/home/pc-146/Documents/UFRJ/8° Período/Otimização/Trabalhos/Trabalho Prático/cos360_optimization_implementation_assignment/objective_function_util.py:7: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.log(prod + 1))


## Apresentação de Resultados

### Método do Gradiente

In [208]:
pd.set_option('display.max_colwidth', None)
print(df_gradiente.head())

                                                                                             Ponto inicial  \
0  [0.5218226579138382, 0.1766351505210949, 0.5230897881307102, 0.006077009838843517, 0.24398081516009718]   
1      [0.5156780040614635, 0.956428224641158, 0.8693971398531677, 0.615504143527244, 0.23086015479926747]   
2     [0.2946226807749026, 0.3898831682375272, 0.2645512612918487, 0.7115753853206658, 0.7818204400208536]   
3   [0.34438551450511534, 0.7289656385061903, 0.35330868392429093, 0.5075578743337829, 0.6734455167549697]   
4    [0.13211349436008835, 0.9989410860061092, 0.8374395095548139, 0.7823695821889023, 0.8345965420536839]   

   # de iterações  # de chamadas de Armijo  \
0             100                      100   
1             100                      100   
2             100                      100   
3             100                      100   
4             100                      100   

                                                              

### Método de Newton

In [209]:
pd.set_option('display.max_colwidth', None)
print(df_newton.head())

                                                                                             Ponto inicial  \
0  [0.5218226579138382, 0.1766351505210949, 0.5230897881307102, 0.006077009838843517, 0.24398081516009718]   
1      [0.5156780040614635, 0.956428224641158, 0.8693971398531677, 0.615504143527244, 0.23086015479926747]   
2     [0.2946226807749026, 0.3898831682375272, 0.2645512612918487, 0.7115753853206658, 0.7818204400208536]   
3   [0.34438551450511534, 0.7289656385061903, 0.35330868392429093, 0.5075578743337829, 0.6734455167549697]   
4    [0.13211349436008835, 0.9989410860061092, 0.8374395095548139, 0.7823695821889023, 0.8345965420536839]   

   # de iterações  \
0               6   
1               7   
2               5   
3              10   
4               8   

                                                                                                    Ponto ótimo  \
0  [0.809839386513708, 0.0011721572056395914, 0.8197387654623859, 1.0554755075053111e-06, 0.00287

### Método Quase-Newton

In [210]:
pd.set_option('display.max_colwidth', None)
print(df_bfgs.head())

                                                                                             Ponto inicial  \
0  [0.5218226579138382, 0.1766351505210949, 0.5230897881307102, 0.006077009838843517, 0.24398081516009718]   
1      [0.5156780040614635, 0.956428224641158, 0.8693971398531677, 0.615504143527244, 0.23086015479926747]   
2     [0.2946226807749026, 0.3898831682375272, 0.2645512612918487, 0.7115753853206658, 0.7818204400208536]   
3   [0.34438551450511534, 0.7289656385061903, 0.35330868392429093, 0.5075578743337829, 0.6734455167549697]   
4    [0.13211349436008835, 0.9989410860061092, 0.8374395095548139, 0.7823695821889023, 0.8345965420536839]   

   # de iterações  \
0               9   
1               9   
2               9   
3               9   
4               9   

                                                                                              Ponto ótimo  \
0      [3.5549992625694857, -4.169509569553837, 1.165139592418136, -2.0886484469361473, 2.7619978231263

# 